# Simulator Snapshots & Analysis Demo

This notebook demonstrates two things:

- Capturing per-step snapshots with in-memory analysis (Jacobians, cycles) from a single engine run.
- Running the `Simulator` with snapshot saving enabled and validating that snapshot artifacts are written to disk.

It uses the built-in `propflow.snapshots` tools (SnapshotManager) integrated into `BPEngine`.


In [1]:
# Setup imports and paths for local src-layout
import sys, json
from pathlib import Path
sys.path.insert(0, 'src')  # ensure local package resolution in editable/dev setup

from propflow.utils import FGBuilder
from propflow.bp.engines import DampingEngine, Engine
from propflow.snapshots import SnapshotsConfig
from propflow.simulator import Simulator

print('Imports OK')

Imports OK


## Part A — In-memory analysis via SnapshotManager

We attach a `SnapshotsConfig` to an engine and run a few steps.
Then we inspect the latest snapshot record for Jacobians, block norms, and basic cycle metrics.


In [2]:
# Build a small cycle factor graph
fg = FGBuilder.build_cycle_graph(
    num_vars=4,
    domain_size=3,
    ct_factory='random_int',
    ct_params={'low': 0, 'high': 5},
    density=1.0,  # unused for cycle, kept for signature
)

# Configure snapshots with analysis enabled (no disk saving here)
snap_cfg = SnapshotsConfig(
    compute_jacobians=True,
    compute_block_norms=True,
    compute_cycles=True,
    include_detailed_cycles=True,
    retain_last=10,
    save_each_step=False,
)

engine = DampingEngine(factor_graph=fg, damping_factor=0.7, snapshots_config=snap_cfg)
for i in range(5):
    engine.step(i)

rec = engine.latest_snapshot()
assert rec is not None, 'No snapshot record captured'
print('Latest step:', rec.data.step)
# Jacobians info
if rec.jacobians is not None:
    print('nQ slots:', len(rec.jacobians.idxQ))
    print('nR slots:', len(rec.jacobians.idxR))
    print('Block norms:', rec.jacobians.block_norms)
else:
    print('No jacobians computed')
# Cycles info
if rec.cycles is not None:
    print('Cycles found:', rec.cycles.num_cycles)
    print('Aligned hops total:', rec.cycles.aligned_hops_total)
    print('Has certified contraction:', rec.cycles.has_certified_contraction)
else:
    print('No cycles metrics computed')

Latest step: 4
nQ slots: 24
nR slots: 24
Block norms: {'||BPA||_inf': 6.666666666666666, '||B||_inf': 1.4285714285714286, '||PA||_inf': 6.666666666666666, '||M||_inf_upper': 3.0}
Cycles found: 2
Aligned hops total: 2
Has certified contraction: False


## Part B — Simulator run with snapshot saving

Here we pass `SnapshotsConfig(save_each_step=True, save_dir=...)` through the `Simulator` engine config.
After a short run, we validate that per-step directories with `meta.json` and `A/P/B.npz` exist.


In [ ]:
# Prepare an output directory under results/ as per repo guidelines
out_dir = Path('results') / 'snapshots_demo'
out_dir.mkdir(parents=True, exist_ok=True)

# (Optional) Clean previous step_* folders for a fresh check
for p in sorted(out_dir.glob('step_*')):
    if p.is_dir():
        for f in p.glob('*'):
            try: f.unlink()
            except Exception: pass
        try: p.rmdir()
        except Exception: pass

# Build a tiny graph list
graphs = [
    FGBuilder.build_cycle_graph(
        num_vars=4, domain_size=3, ct_factory='random_int', ct_params={'low':0,'high':4}, density=1.0
    )
]

# Engine configuration for Simulator, including snapshots saving
engine_configs = {
    'DampingEngine': {
        'class': DampingEngine,
        'damping_factor': 0.7,
        'snapshots_config': SnapshotsConfig(
            compute_jacobians=True,
            compute_block_norms=True,
            compute_cycles=True,
            include_detailed_cycles=False,
            retain_last=5,
            save_each_step=True,
            save_dir=str(out_dir),
        ),
    }
}

sim = Simulator(engine_configs, log_level='MINIMAL')
_ = sim.run_simulations(graphs, max_iter=4)

# Validate saved artifacts
step_dirs = sorted(out_dir.glob('step_*'))
print('Saved steps:', [p.name for p in step_dirs])
assert step_dirs, 'No step_* directories found; snapshots may not have been saved'

first = step_dirs[0]
assert (first / 'meta.json').exists(), 'meta.json missing'
assert (first / 'A.npz').exists(), 'A.npz missing'
assert (first / 'P.npz').exists(), 'P.npz missing'
assert (first / 'B.npz').exists(), 'B.npz missing'

with (first / 'meta.json').open() as fh:
    meta = json.load(fh)
print('meta keys:', sorted(meta.keys()))
print('cycles summary:', meta.get('cycles', {}))
print('block norms:', meta.get('block_norms'))
print('OK — snapshot files present and readable.')